In [2]:
import os
import pandas as pd

os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data1 = pd.read_csv('0726_masking_labeling_data_0.csv')
data2 = pd.read_csv('0726_masking_labeling_data_1.csv')


selected_columns1 = ['text_morphed', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성 민원']
data1 = data1[selected_columns1]
data2 = data2[selected_columns1]

data2['욕설_모욕'].value_counts()

욕설_모욕
0    332
1     34
Name: count, dtype: int64

In [ ]:
data2['저격성민원_'].value_counts()

In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
import joblib

# 데이터 불러오기
os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data1 = pd.read_csv('0726_masking_labeling_data_0.csv')
data2 = pd.read_csv('0726_masking_labeling_data_1.csv')

data1 = data1.rename(columns={'text_morphed':'text_morphed_'})
data2 = data2.rename(columns={'text_morphed':'text_morphed_'})


selected_columns1 = ['title','text_morphed_', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성 민원']

target = selected_columns1[2]

data1 = data1[selected_columns1]
data2 = data2[selected_columns1]
data1 = data1[300:334]                    #라벨 데이터 개수만큼 설정
data2 = data2[data2[target] == 1]   #라벨 컬럼 설정

data3 = pd.concat([data1, data2])
# 텍스트와 라벨 분리
data_contents = data3['text_morphed_'] + ' ' + data3['title']
data_labeling = data3[f'{target}']    #라벨 컬럼 설정

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_contents, data_labeling, stratify=data_labeling, test_size=0.2,)

# 파이프라인 정의
pipelines = {

    'extra_trees': Pipeline([
        ('vect', TfidfVectorizer()),
        ('et', ExtraTreesClassifier())
    ]),

    'ridge_classifier': Pipeline([
        ('vect', TfidfVectorizer()),
        ('ridge', RidgeClassifier())
    ]),

    'logistic_regression': Pipeline([
        ('vect', TfidfVectorizer()),
        ('lr', LogisticRegression())
    ]),

    'naive_bayes': Pipeline([
        ('vect', TfidfVectorizer()),
        ('nb', MultinomialNB())
    ]),

    'svm': Pipeline([
        ('vect', TfidfVectorizer()),
        ('svm', SVC())
    ]),


    'hard_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('hard_model',VotingClassifier([('LR',LogisticRegression()),('ridge',RidgeClassifier()),('svm',SVC()),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='hard'))

    ]),
    'soft_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('soft_model',VotingClassifier([('LR',LogisticRegression()),('svm',SVC(probability=True)),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='soft'))

    ])
    

}

# 모델 훈련 및 평가
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")

    # 텍스트 데이터를 TF-IDF 벡터로 변환
    X_train_tfidf = pipeline.named_steps['vect'].fit_transform(X_train)
    X_test_tfidf = pipeline.named_steps['vect'].transform(X_test)
    
    # 차원 출력
    print(f"{model_name} - X_train shape: {X_train_tfidf.shape}")
    print(f"{model_name} - X_test shape: {X_test_tfidf.shape}")
    
    # 모델 훈련
    pipeline.fit(X_train, y_train)
    
    # 교차검증 점수 계산
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    
    # 성능 출력
    train_score = pipeline.score(X_train, y_train)
    test_score = pipeline.score(X_test, y_test)
    print(f"{model_name} Train set score: {train_score:.4f}")
    print(f"{model_name} Test set score: {test_score:.4f}")
    print(f"{model_name} Cross-validation scores: {cv_scores}")
    print(f"{model_name} Cross-validation mean score: {cv_scores.mean():.4f}\n")
    
    # 모델 저장
    model_save_path = os.path.join('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/', f'{target}_{model_name}_tfidf_nonemasking_model.pkl')
    joblib.dump(pipeline, model_save_path)

print("All models have been trained and saved.")

Training extra_trees...
extra_trees - X_train shape: (320, 7417)
extra_trees - X_test shape: (80, 7417)
extra_trees Train set score: 1.0000
extra_trees Test set score: 0.9125
extra_trees Cross-validation scores: [0.90625  0.90625  0.921875 0.921875 0.921875]
extra_trees Cross-validation mean score: 0.9156

Training ridge_classifier...
ridge_classifier - X_train shape: (320, 7417)
ridge_classifier - X_test shape: (80, 7417)
ridge_classifier Train set score: 0.9938
ridge_classifier Test set score: 0.9125
ridge_classifier Cross-validation scores: [0.90625  0.90625  0.921875 0.921875 0.921875]
ridge_classifier Cross-validation mean score: 0.9156

Training logistic_regression...
logistic_regression - X_train shape: (320, 7417)
logistic_regression - X_test shape: (80, 7417)
logistic_regression Train set score: 0.9156
logistic_regression Test set score: 0.9125
logistic_regression Cross-validation scores: [0.90625  0.90625  0.921875 0.921875 0.921875]
logistic_regression Cross-validation mean 

In [9]:
data2

,text_morphed_masked_,aggr,욕설_모욕,비꼼_시비,성희롱,요지불명,저격성민원_
14,저번 국민 신문고 광산경찰서 올리 접수 이첩 하 기억 날 다시 `지역` 경찰서 올리...,1,1,1,0,1,0
20,`지역` `지역` 피버노바 고시원 난장이 목수 놈 주제 재판 변호사비 날리 ㄱ ㅡ ...,1,1,1,0,1,0
28,얼마 전 차량 운행 라지에 터 파열 되 노상 정차 일 생기 `지역` 주황색 점선 도...,1,1,1,0,0,0
35,안녕 하 우선 간접흡연 인하 `지역` 나쁘 두서 없 죄송 무슨 부산대역 근방 전체 ...,1,1,1,0,0,0
44,`지역` `지역` 세대 에스 아파트 아파트 관리실 약 전 바로 옆 구 아 촌 건물 ...,1,1,1,0,0,0
85,`지역` 청 담당자 아래 같이 지나 민원 시작 부 분명히 각자 민원인 제기 의견 다...,1,1,1,0,0,0
89,귀 의심 어떻하 고등학교 앞 세대 아파트 옆 높이 철탑 세우 밤낮없이 공치 소리 날...,1,1,1,0,0,0
90,도데체 무슨 생각 허가 하 한심 진짜 주말 새벽 골프 공 소리 땅땅 하 들리 잠 깨...,1,1,1,0,0,0
93,골프장 짓 지금 포크레인 땅 헤집 난리 발굴 조사 바로 몇 미터 옆 친환경 생태 사...,1,1,1,0,0,0
94,`지역` 암암리 `지역` 바로 위 기괴 철탑 거대 하 공용 야외 골프장 허기 하 하...,1,1,1,0,0,0


In [ ]:
selected_columns1 = ['text_morphed_masked_', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성민원_']
data1 = data1[selected_columns1]
data2 = data2[selected_columns1]
data1 = data1[:22]
data2 = data2[data2.욕설_모욕 == 1]

In [50]:

# 데이터 병합
data3 = pd.concat([data1, data2])

In [62]:
data3['욕설_모욕'].value_counts()

욕설_모욕
0    374
1     21
Name: count, dtype: int64

In [4]:
data2['욕설_모욕'].value_counts()

욕설_모욕
0    176
1     21
Name: count, dtype: int64

In [14]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
import joblib

# 데이터 불러오기
os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data1 = pd.read_csv('0726_masking_labeling_data_0.csv')
data2 = pd.read_csv('0726_masking_labeling_data_1.csv')

data1 = data1.rename(columns={'text_morphed':'text_morphed_'})
data2 = data2.rename(columns={'text_morphed':'text_morphed_'})

data1 = data1.rename(columns={'저격성 민원':'저격성민원_'})
data2 = data2.rename(columns={'저격성 민원':'저격성민원_'})

selected_columns1 = ['text_morphed_', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성민원_']
data1 = data1[selected_columns1]
data2 = data2[selected_columns1]
data1 = data1[:366]
data2 = data2[data2.aggr == 1]

data3 = pd.concat([data1, data2])
# 텍스트와 라벨 분리
data_contents = data3['text_morphed_']
data_labeling = data3['aggr']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_contents, data_labeling, stratify=data_labeling, test_size=0.2, random_state=42)

# 파이프라인 정의
pipelines = {

    'extra_trees': Pipeline([
        ('vect', TfidfVectorizer()),
        ('et', ExtraTreesClassifier())
    ]),

    'ridge_classifier': Pipeline([
        ('vect', TfidfVectorizer()),
        ('ridge', RidgeClassifier())
    ]),

    'logistic_regression': Pipeline([
        ('vect', TfidfVectorizer()),
        ('lr', LogisticRegression())
    ]),

    'naive_bayes': Pipeline([
        ('vect', TfidfVectorizer()),
        ('nb', MultinomialNB())
    ]),

    'svm': Pipeline([
        ('vect', TfidfVectorizer()),
        ('svm', SVC())
    ]),


    'hard_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('hard_model',VotingClassifier([('LR',LogisticRegression()),('ridge',RidgeClassifier()),('svm',SVC())],voting='hard'))

    ])
    

}

# 모델 훈련 및 평가
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")

    # 텍스트 데이터를 TF-IDF 벡터로 변환
    X_train_tfidf = pipeline.named_steps['vect'].fit_transform(X_train)
    X_test_tfidf = pipeline.named_steps['vect'].transform(X_test)
    
    # 차원 출력
    print(f"{model_name} - X_train shape: {X_train_tfidf.shape}")
    print(f"{model_name} - X_test shape: {X_test_tfidf.shape}")
    
    # 모델 훈련
    pipeline.fit(X_train, y_train)
    
    # 교차검증 점수 계산
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    
    # 성능 출력
    train_score = pipeline.score(X_train, y_train)
    test_score = pipeline.score(X_test, y_test)
    print(f"{model_name} Train set score: {train_score:.4f}")
    print(f"{model_name} Test set score: {test_score:.4f}")
    print(f"{model_name} Cross-validation scores: {cv_scores}")
    print(f"{model_name} Cross-validation mean score: {cv_scores.mean():.4f}\n")
    
    # 모델 저장
    model_save_path = os.path.join('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/', f'욕설_모욕_{model_name}_tfidf_nonemasking_model.pkl')
    joblib.dump(pipeline, model_save_path)

print("All models have been trained and saved.")

Training logistic_regression...
Fitting 3 folds for each of 90 candidates, totalling 270 fits


c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_


logistic_regression Best parameters found: {'lr__C': 0.01, 'lr__solver': 'liblinear', 'pca__n_components': 100, 'vect__max_features': 10000}
logistic_regression Best cross-validation score: 0.8283
logistic_regression Test set score: 0.6667

Training svm...
Fitting 3 folds for each of 90 candidates, totalling 270 fits


c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_


svm Best parameters found: {'pca__n_components': 100, 'svm__C': 0.1, 'svm__kernel': 'linear', 'vect__max_features': 10000}
svm Best cross-validation score: 0.7045
svm Test set score: 0.5556

Training random_forest...
Fitting 3 folds for each of 108 candidates, totalling 324 fits


...

c:\Users\USER\anaconda3\Lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
81 fits failed out of a total of 729.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
81 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\pip


xgboost Best parameters found: {'pca__n_components': 100, 'vect__max_features': 10000, 'xgb__learning_rate': 0.01, 'xgb__max_depth': 3, 'xgb__n_estimators': 50}
xgboost Best cross-validation score: 0.5581
xgboost Test set score: 0.6667

Training lightgbm...
Fitting 3 folds for each of 243 candidates, totalling 729 fits


c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
81 fits failed out of a total of 729.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
81 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 17, number of negative: 17
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 34, number of used features: 0
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirement

c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
81 fits failed out of a total of 729.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
81 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 359, in _fit
    X, fitted_transformer = fit_transform_

In [66]:
data_contents

0      안녕 하 관리자 `지역` 문화센터 인근 학교 재직 교사 `지역` 문화센터 근방 신호...
1      안녕 하 `지역` 아이 주민 에 인도 파손 되 넘어지 경험 있 인도 파손 부분 인도...
2      `지역` `지역` `지역` 차선 도로 파손 되 매우 크 불편 겪 사진 첨부 지하철 ...
3      `지역` 교차로 비보 호 좌회전 신호 있 좌회전 신호일 때 좌회전 도로 횡단보도 신...
4      안녕 하 코로나 도둑 맞 같 현실 모든 가족 너무 화 세상에 있 답답 일단 국민 신...
                             ...                        
192    `지역` 빌 쓰레기 처리 대하 건의 사람 알 빌라 주민 만나 옮기 주시 하 궁 나 ...
193    추석연 휴 친정 이틀 `지역` 빌라 차이 아직 덥 때 문 열 미치 알 사람 버리 쓰...
194                 지나 달 차례 문의 드리 아직 방지 턱 설치 안 되 있 지나 치이
195    군수 건축 과장 안녕 하 지나 일요일 오전 군청 발주 하 `지역` 농공 단지 앞 하...
196    군도 호선 미개 설 공 촉구 건 대하 부여군 건설과 도로계 대답 시원 보충 질문 존...
Name: text_morphed_masked_, Length: 395, dtype: object